In [1]:
import ee
import geemap
import os
from IPython.display import display

# Initialize Earth Engine
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Define area of interest
aoi = ee.Geometry.Polygon([[[-45.559114, -18.954365], [-45.559114, -18.212409], 
                           [-44.839706, -18.212409], [-44.839706, -18.954365], 
                           [-45.559114, -18.954365]]])

# Read min and max values from stats file
stats_file_path = 'chlorophyll_analysis/Tres_Marias_20200101_stats.txt'
with open(stats_file_path, 'r') as f:
    lines = f.readlines()
    min_value = float(lines[2].split(': ')[1])
    max_value = float(lines[3].split(': ')[1])

print(f"Chlorophyll concentration range:")
print(f"Minimum: {min_value:.2f}")
print(f"Maximum: {max_value:.2f}")

# Create and setup the map
Map = geemap.Map()
Map.centerObject(aoi, zoom=10)
Map.add_basemap('SATELLITE')

# Add the GeoTIFF directly to the map
geotiff_path = 'chlorophyll_analysis/Tres_Marias_20200101_chlorophyll.tif'

# Add raster layer - the -9999 values from the AI processing will be treated as nodata
Map.add_raster(
    geotiff_path,
    palette=[
        '#0000ff', '#00ffff', '#00ff00', '#ffff00', '#ff7f00', '#ff0000',
        '#8b0000', '#800080', '#ff00ff', '#8b4513', '#000000'
    ],
    vmin=min_value,
    vmax=max_value,
    nodata=-9999,  # This will hide all non-water pixels that were marked as -9999
    layer_name='Predicted Chlorophyll'
)

Map.addLayer(
    aoi, 
    {'color': 'white', 'width': 2, 'fillColor': '00000000'}, 
    'AOI Boundary'
)

Map.addLayerControl()

# Function to add legend
def add_legend(map_obj, title, palette, min_value, max_value):
    legend_html = f"""
    <div style='padding: 10px; background-color: white; border-radius: 5px;'>
        <h4>{title}</h4>
        <div style='display: flex; align-items: center;'>
            <span>{min_value:.2f}</span>
            <div style='flex-grow: 1; height: 20px; background: linear-gradient(to right, {", ".join(palette)}); margin: 0 10px;'></div>
            <span>{max_value:.2f}</span>
        </div>
    </div>
    """
    map_obj.add_html(legend_html)

# Add legend
add_legend(
    Map, 
    'Predicted Chlorophyll',
    [
        '#0000ff', '#00ffff', '#00ff00', '#ffff00', '#ff7f00', '#ff0000',
        '#8b0000', '#800080', '#ff00ff', '#8b4513', '#000000'
    ],
    min_value,
    max_value
)

# Display the map
display(Map)

Chlorophyll concentration range:
Minimum: -0.34
Maximum: 6.53


Map(center=[-18.58345884758661, -45.1994100000001], controls=(WidgetControl(options=['position', 'transparent_…